## 第2章 搜索问题

### 2.1 DNA搜索

In [1]:
from enum import IntEnum
from typing import Tuple, List

Nucleotide: IntEnum = IntEnum('Nucleotide', ('A', 'C', 'G', 'T'))
Codon = Tuple[Nucleotide, Nucleotide, Nucleotide]  # type alias for codons
Gene = List[Codon]  # type alias for genes
gene_str: str = "ACGTGGCTCTCTAACGTACGTACGTACGGGGTTTATATATACCCTAGGACTCCCTTT"

def string_to_gene(s: str) -> Gene:
    gene: Gene = []
    for i in range(0, len(s), 3):
        if (i + 2) >= len(s):  # don't run off end!
            return gene
        #  initialize codon out of three nucleotides
        codon: Codon = (Nucleotide[s[i]], Nucleotide[s[i + 1]], Nucleotide[s[i + 2]])
        gene.append(codon)  # add codon to gene
    return gene

In [2]:
my_gene: Gene = string_to_gene(gene_str)

def linear_contains(gene: Gene, key_codon: Codon) -> bool:
    for codon in gene:
        if codon == key_codon:
            return True
    return False

acg: Codon = (Nucleotide.A, Nucleotide.C, Nucleotide.G)
gat: Codon = (Nucleotide.G, Nucleotide.A, Nucleotide.T)
print(linear_contains(my_gene, acg))  # True
print(linear_contains(my_gene, gat))  # False

True
False


In [3]:
def binary_contains(gene: Gene, key_codon: Codon) -> bool:
    low: int = 0
    high: int = len(gene) - 1
    while low <= high:  # while there is still a search space
        mid: int = (low + high) // 2
        if gene[mid] < key_codon:
            low = mid + 1
        elif gene[mid] > key_codon:
            high = mid - 1
        else:
            return True
    return False

my_sorted_gene: Gene = sorted(my_gene)
print(binary_contains(my_sorted_gene, acg))  # True
print(binary_contains(my_sorted_gene, gat))  # False

True
False


In [4]:
from __future__ import annotations
from typing import TypeVar, Iterable, Sequence, Generic, List, Callable, Set, Deque, Dict, Any, Optional, Protocol
#from typing_extensions import Protocol
from heapq import heappush, heappop

T = TypeVar('T')

def linear_contains(iterable: Iterable[T], key: T) -> bool:
    for item in iterable:
        if item == key:
            return True
    return False

C = TypeVar("C", bound="Comparable")

class Comparable(Protocol):
    def __eq__(self, other: Any) -> bool:
       ...
    def __lt__(self: C, other: C) -> bool:
       ...
    def __gt__(self: C, other: C) -> bool:
        return (not self < other) and self != other

    def __le__(self: C, other: C) -> bool:
        return self < other or self == other

    def __ge__(self: C, other: C) -> bool:
        return not self < other

def binary_contains(sequence: Sequence[C], key: C) -> bool:
    low: int = 0
    high: int = len(sequence) - 1
    while low <= high:  # while there is still a search space
        mid: int = (low + high) // 2
        if sequence[mid] < key:
            low = mid + 1
        elif sequence[mid] > key:
            high = mid - 1
        else:
            return True
    return False


print(linear_contains([1, 5, 15, 15, 15, 15, 20], 5))  # True
print(binary_contains(["a", "d", "e", "f", "z"], "f"))  # True
print(binary_contains(["john", "mark", "ronald", "sarah"], "sheila"))  # False

True
True
False


### 2.2 求解迷宫问题

下面代码引用了[generic_search模块](https://github.com/WillKoehrsen/classic_computer_science)

In [7]:
# Maze.py
from enum import Enum
from typing import List, NamedTuple, Callable, Optional
import random
from math import sqrt

class Cell(str, Enum):
    EMPTY = " "
    BLOCKED = "X"
    START = "S"
    GOAL = "G"
    PATH = "*"

class MazeLocation(NamedTuple):
    row: int
    column: int

class Maze:
    def __init__(self, rows: int = 10, columns: int = 10, sparseness: float = 0.2, start: \
        MazeLocation = MazeLocation(0, 0), goal: MazeLocation = MazeLocation(9, 9)) -> None:
        # initialize basic instance variables
        self._rows: int = rows
        self._columns: int = columns
        self.start: MazeLocation = start
        self.goal: MazeLocation = goal
        # fill the grid with empty cells
        self._grid: List[List[Cell]] = [[Cell.EMPTY for c in range(columns)] 
    for r in range(rows)]
        # populate the grid with blocked cells
        self._randomly_fill(rows, columns, sparseness)
        # fill the start and goal locations in
        self._grid[start.row][start.column] = Cell.START
        self._grid[goal.row][goal.column] = Cell.GOAL

    def _randomly_fill(self, rows: int, columns: int, sparseness: float):
        for row in range(rows):
            for column in range(columns):
                if random.uniform(0, 1.0) < sparseness:
                    self._grid[row][column] = Cell.BLOCKED
    # return a nicely formatted version of the maze for printing
    def __str__(self) -> str:
        output: str = ""
        for row in self._grid:
            output += "".join([c.value for c in row]) + "\n"
        return output
    def goal_test(self, ml: MazeLocation) -> bool:
        return ml == self.goal
    def successors(self, ml: MazeLocation) -> List[MazeLocation]:
        locations: List[MazeLocation] = []
        if ml.row + 1 < self._rows and self._grid[ml.row + 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row + 1, ml.column))
        if ml.row - 1 >= 0 and self._grid[ml.row - 1][ml.column] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row - 1, ml.column))
        if ml.column + 1 < self._columns and self._grid[ml.row][ml.column + 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column + 1))
        if ml.column - 1 >= 0 and self._grid[ml.row][ml.column - 1] != Cell.BLOCKED:
            locations.append(MazeLocation(ml.row, ml.column - 1))
        return locations
    
    def mark(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.PATH
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL

    def clear(self, path: List[MazeLocation]):
        for maze_location in path:
            self._grid[maze_location.row][maze_location.column] = Cell.EMPTY
        self._grid[self.start.row][self.start.column] = Cell.START
        self._grid[self.goal.row][self.goal.column] = Cell.GOAL
        
def euclidean_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml: MazeLocation) -> float:
        xdist: int = ml.column - goal.column
        ydist: int = ml.row - goal.row
        return sqrt((xdist * xdist) + (ydist * ydist))
    return distance

def manhattan_distance(goal: MazeLocation) -> Callable[[MazeLocation], float]:
    def distance(ml: MazeLocation) -> float:
        xdist: int = abs(ml.column - goal.column)
        ydist: int = abs(ml.row - goal.row)
        return (xdist + ydist)
    return distance

In [11]:
from generic_search import dfs, node_to_path, Node
from time import perf_counter as tic

# Test DFS
m: Maze = Maze(6, 10, 0.2, MazeLocation(0, 0), MazeLocation(5, 9))
print(m)
print("路径解算中...")
st = tic()
solution1: Optional[Node[MazeLocation]] = dfs(m.start, m.goal_test, m.successors)
print("路径解算历时: {:5f}s".format(tic() - st))
if solution1 is None:
    print("No solution found using depth-first search!")
else:
    path1: List[MazeLocation] = node_to_path(solution1)
    m.mark(path1)
    print(m)
    m.clear(path1)

S X    XX 
  X   X X 
          
     X   X
     XX   
X      X G

路径解算中...
路径解算历时: 0.000534s
S*X    XX 
 *X   X X 
**  ***** 
*   *X  *X
*****XX **
X      X G



In [12]:
from generic_search import bfs
from time import perf_counter as tic

# Test BFS
print("路径解算中...")
st = tic()
solution2: Optional[Node[MazeLocation]] = bfs(m.start, m.goal_test, m.successors)
print("路径解算历时: {:5f}s".format(tic() - st))
if solution2 is None:
    print("No solution found using breadth-first search!")
else:
    path2: List[MazeLocation] = node_to_path(solution2)
    m.mark(path2)
    print(m)
    m.clear(path2)

路径解算中...
路径解算历时: 0.001139s
S X    XX 
* X   X X 
*******   
     X** X
     XX** 
X      X*G



In [13]:
from generic_search import astar
from time import perf_counter as tic
# Test A*
distance: Callable[[MazeLocation], float] = manhattan_distance(m.goal)
print("路径解算中...")
st = tic()
solution3: Optional[Node[MazeLocation]] = astar(m.start, m.goal_test, m.successors, distance)
print("路径解算历时: {:5f}s".format(tic() - st))
if solution3 is None:
    print("No solution found using A*!")
else:
    path3: List[MazeLocation] = node_to_path(solution3)
    m.mark(path3)
    print(m)

路径解算中...
路径解算历时: 0.001013s
S X    XX 
* X   X X 
*******   
     X** X
     XX** 
X      X*G



### 2.3 传教士和食人族

In [27]:
from __future__ import annotations
from typing import List, Optional
from generic_search import bfs, Node, node_to_path

MAX_NUM: int = 3

class MCState:
    def __init__(self, missionaries: int, cannibals: int, boat: bool) -> None:
        self.wm: int = missionaries # west bank missionaries
        self.wc: int = cannibals # west bank cannibals
        self.em: int = MAX_NUM - self.wm  # east bank missionaries
        self.ec: int = MAX_NUM - self.wc  # east bank cannibals
        self.boat: bool = boat

    def __str__(self) -> str:
        return (f"西岸 有{self.wm}个传教士和{self.wc}个食人族.\n" 
                f"东岸 有{self.em}个传教士和{self.ec}个食人族.\n"
                f"小船在 {'西' if self.boat else '东'}岸.")
    
    def goal_test(self) -> bool:
        return self.is_legal and self.em == MAX_NUM and self.ec == MAX_NUM
    @property
    def is_legal(self) -> bool:
        if self.wm < self.wc and self.wm > 0:
            return False
        if self.em < self.ec and self.em > 0:
            return False
        return True
    def successors(self) -> List[MCState]:
        sucs: List[MCState] = []
        if self.boat: # boat on west bank
            if self.wm > 1:
                sucs.append(MCState(self.wm - 2, self.wc, not self.boat))
            if self.wm > 0:
                sucs.append(MCState(self.wm - 1, self.wc, not self.boat))
            if self.wc > 1:
                sucs.append(MCState(self.wm, self.wc - 2, not self.boat))
            if self.wc > 0:
                sucs.append(MCState(self.wm, self.wc - 1, not self.boat))
            if (self.wc > 0) and (self.wm > 0):
                sucs.append(MCState(self.wm - 1, self.wc - 1, not self.boat))
        else: # boat on east bank
            if self.em > 1:
                sucs.append(MCState(self.wm + 2, self.wc, not self.boat))
            if self.em > 0:
                sucs.append(MCState(self.wm + 1, self.wc, not self.boat))
            if self.ec > 1:
                sucs.append(MCState(self.wm, self.wc + 2, not self.boat))
            if self.ec > 0:
                sucs.append(MCState(self.wm, self.wc + 1, not self.boat))
            if (self.ec > 0) and (self.em > 0):
                sucs.append(MCState(self.wm + 1, self.wc + 1, not self.boat))
        return [x for x in sucs if x.is_legal]
    
def display_solution(path: List[MCState]):
    if len(path) == 0: # sanity check
        return
    old_state: MCState = path[0]
    print(old_state)
    for current_state in path[1:]:
        if current_state.boat:
            print(f"{old_state.em - current_state.em} 个传教士和 " +
                  f"{old_state.ec - current_state.ec} 个食人族" +
                  " 东岸 >>>...西岸.\n")
        else:
            print(f"{old_state.wm - current_state.wm} 个传教士和 " +
                  f"{old_state.wc -current_state.wc} 个食人族" +
                  " 西岸 >>>...东岸.\n")
        print(current_state)
        old_state = current_state
        
start: MCState = MCState(MAX_NUM, MAX_NUM, True)
solution: Optional[Node[MCState]] = bfs(start, MCState.goal_test, MCState.successors)
if solution is None:
    print("无解!")
else:
    path: List[MCState] = node_to_path(solution)
    display_solution(path)

西岸 有3个传教士和3个食人族.
东岸 有0个传教士和0个食人族.
小船在 西岸.
0 个传教士和 2 个食人族 西岸 >>>...东岸.

西岸 有3个传教士和1个食人族.
东岸 有0个传教士和2个食人族.
小船在 东岸.
0 个传教士和 1 个食人族 东岸 >>>...西岸.

西岸 有3个传教士和2个食人族.
东岸 有0个传教士和1个食人族.
小船在 西岸.
0 个传教士和 2 个食人族 西岸 >>>...东岸.

西岸 有3个传教士和0个食人族.
东岸 有0个传教士和3个食人族.
小船在 东岸.
0 个传教士和 1 个食人族 东岸 >>>...西岸.

西岸 有3个传教士和1个食人族.
东岸 有0个传教士和2个食人族.
小船在 西岸.
2 个传教士和 0 个食人族 西岸 >>>...东岸.

西岸 有1个传教士和1个食人族.
东岸 有2个传教士和2个食人族.
小船在 东岸.
1 个传教士和 1 个食人族 东岸 >>>...西岸.

西岸 有2个传教士和2个食人族.
东岸 有1个传教士和1个食人族.
小船在 西岸.
2 个传教士和 0 个食人族 西岸 >>>...东岸.

西岸 有0个传教士和2个食人族.
东岸 有3个传教士和1个食人族.
小船在 东岸.
0 个传教士和 1 个食人族 东岸 >>>...西岸.

西岸 有0个传教士和3个食人族.
东岸 有3个传教士和0个食人族.
小船在 西岸.
0 个传教士和 2 个食人族 西岸 >>>...东岸.

西岸 有0个传教士和1个食人族.
东岸 有3个传教士和2个食人族.
小船在 东岸.
1 个传教士和 0 个食人族 东岸 >>>...西岸.

西岸 有1个传教士和1个食人族.
东岸 有2个传教士和2个食人族.
小船在 西岸.
1 个传教士和 1 个食人族 西岸 >>>...东岸.

西岸 有0个传教士和0个食人族.
东岸 有3个传教士和3个食人族.
小船在 东岸.
